In [1]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE96nnn/GSE96107/suppl/GSE96107_ES_Ring1B.IDR0.05.filt.narrowPeak.gz -q

In [2]:
!gunzip GSE96107_ES_Ring1B.IDR0.05.filt.narrowPeak.gz

In [5]:
!bedtools sort -i GSE96107_ES_Ring1B.IDR0.05.filt.narrowPeak | bedtools merge \
> GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed

In [19]:
!for f in CAP-CGI_CpG_density_quartile_?_mm10_merged5kb.bed; do bedtools intersect \
-a $f -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed -wa > \
"${f/.bed/_RING1B_overlap.bed}"; done

In [20]:
!for f in CAP-CGI_CpG_density_quartile_?_mm10_merged5kb.bed; do bedtools intersect -v \
-a $f -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed > \
"${f/.bed/_noRING1B.bed}"; done

In [9]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/genes/mm10.refGene.gtf.gz -q
!gunzip mm10.refGene.gtf.gz

In [35]:
!awk '{if($1!~"random") {print $1,$4,$5,$7,$10,$14,$3}}' mm10.refGene.gtf | \
awk '{gsub(";",""); print}' | awk '{gsub(/"/, ""); print}' | \
awk '{if($7=="exon") {print $1"\t"$2"\t"$3"\t"$5"\t"$6"\t"$4}}' > refGene_exons_mm10.bed

In [40]:
import pandas as pd
exons = pd.read_csv("refGene_exons_mm10.bed", sep="\t", header=None)
exons.columns = ["chrom", "start", "end", "gene", "exon", "strand"]
exons["max_exon"] = exons.groupby(["gene", "strand"])["exon"].transform('max')

In [61]:
first_exon = exons.loc[((exons["strand"] == "+") & (exons["exon"] == 1)) |
                       ((exons["strand"] == "-") & (exons["exon"] == exons["max_exon"])),
                       ["chrom", "start", "end", "gene", "exon", "strand"]].drop_duplicates()
first_exon = first_exon.sort_values(["chrom", "start", "end"]).drop_duplicates(subset="gene")

In [69]:
first_exon.to_csv("mm10.refGene_exon1.bed", sep="\t", header=None, index=False)

In [70]:
!for f in ESC_4SU_refGene_exon1_quartile_?_TSS_mm10.bed; do bedtools intersect -v \
-a $f -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed > \
"${f/.bed/_noRING1B.bed}"; done

In [76]:
!for f in ESC_4SU_refGene_exon1_quartile_?_TSS_mm10_noRING1B.bed; \
do bedtools intersect -v -a $f -b CAP-CGI_mm10.bed -wa | wc -l; done

4640
3884
1959
732


In [78]:
!for f in ESC_4SU_refGene_exon1_quartile_?_TSS_mm10_noRING1B.bed; \
do bedtools intersect -v -a $f -b CAP-CGI_mm10.bed -wa | \
shuf -n 732 > "${f/.bed/_noCGI_shuf732.bed}"; done

In [79]:
!for f in ESC_4SU_refGene_exon1_quartile_?_TSS_mm10_noRING1B.bed; \
do bedtools intersect -a $f -b CAP-CGI_mm10.bed -wa | \
shuf -n 732 > "${f/.bed/_CGI_shuf732.bed}"; done

In [80]:
!awk -v OFS="\t" '{print $1,$2,$3,$4,$5,"-"}' \
ESC_4SU_refGene_exon1_quartile_1_TSS_mm10_noRING1B.bed > \
ESC_4SU_refGene_exon1_quartile_1_TSS_mm10_noRING1B_minus.bed
!awk -v OFS="\t" '{print $1,$2,$3,$4,$5,"-"}' \
ESC_4SU_refGene_exon1_quartile_2_TSS_mm10_noRING1B.bed > \
ESC_4SU_refGene_exon1_quartile_2_TSS_mm10_noRING1B_minus.bed
!awk -v OFS="\t" '{print $1,$2,$3,$4,$5,"-"}' \
ESC_4SU_refGene_exon1_quartile_3_TSS_mm10_noRING1B.bed > \
ESC_4SU_refGene_exon1_quartile_3_TSS_mm10_noRING1B_minus.bed
!awk -v OFS="\t" '{print $1,$2,$3,$4,$5,"+"}' \
ESC_4SU_refGene_exon1_quartile_4_TSS_mm10_noRING1B.bed > \
ESC_4SU_refGene_exon1_quartile_4_TSS_mm10_noRING1B_plus.bed

In [82]:
!for f in ESC_4SU_refGene_exon1_quartile_?_TSS_mm10_noRING1B_minus.bed; \
do cat $f ESC_4SU_refGene_exon1_quartile_4_TSS_mm10_noRING1B_plus.bed \
| bedtools sort -i > "${f/.bed/_quartile_4_plus.bed}";done

In [85]:
!grep GSE102517.KLF4.mESC \
/exports/igmm/eddie/wendy-lab/elias/Friman2022/Peaks/mm10/CRE/remap2022_all_macs2_mm10_v1_0.bed \
| bedtools sort -i | bedtools merge > GSE102517_KLF4_mESC_remap.bed

In [86]:
!grep GSE24029.RAD21.mESC \
/exports/igmm/eddie/wendy-lab/elias/Friman2022/Peaks/mm10/CRE/remap2022_all_macs2_mm10_v1_0.bed \
| bedtools sort -i | bedtools merge > GSE24029_RAD21_mESC_remap.bed

In [87]:
!grep GSE41314.RNF2.mESC \
/exports/igmm/eddie/wendy-lab/elias/Friman2022/Peaks/mm10/CRE/remap2022_all_macs2_mm10_v1_0.bed \
| bedtools sort -i | bedtools merge > GSE41314_RNF2_mESC_remap.bed

In [89]:
!bedtools sort -i /exports/igmm/eddie/wendy-lab/ilia/mouse_factor/82185_sort_peaks.narrowPeak.bed \
| bedtools merge > GSE93536_CXXC1_mESCS_cistromeDB.bed

In [90]:
!bedtools intersect -v -a GSE102517_KLF4_mESC_remap.bed -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed \
> GSE102517_KLF4_mESC_remap_noRING1B.bed

In [91]:
!bedtools intersect -v -a GSE93536_CXXC1_mESCS_cistromeDB.bed -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed \
> GSE93536_CXXC1_mESCS_cistromeDB_noRING1B.bed

In [93]:
!cat ESC_4SU_refGene_exon1_quartile_?_TSS_mm10.bed | bedtools sort -i > mm10.refGene_exon1_TSS.bed

In [95]:
!bedtools intersect -v -a mm10.refGene_exon1_TSS.bed -b GSE96107_ES_Ring1B.IDR0.05.filt_merged.bed \
| bedtools sort -i > mm10.refGene_exon1_TSS_noRING1B.bed 